In [ ]:
class KnowledgeBase:
    def __init__(self):  # Corrected constructor
        self.diseases = {
            "Common Cold": {"symptoms": ["sneezing", "runny_nose"], "treatment": "rest and fluids", "severity": "mild"},
            "Flu": {"symptoms": ["fever", "cough", "body_aches"], "treatment": "rest and fluids", "severity": "moderate"},
            "Pneumonia": {"symptoms": ["fever", "cough", "shortness_of_breath"], "treatment": "antibiotics", "severity": "severe"},
            "Tuberculosis": {"symptoms": ["cough", "weight_loss", "night_sweats"], "treatment": "antibiotics", "severity": "severe"},
            "COVID-19": {"symptoms": ["fever", "cough", "shortness_of_breath", "loss_of_taste", "fatigue"], 
                         "treatment": "quarantine, fluids, and rest", "severity": "moderate"},
            "Allergy": {"symptoms": ["sneezing", "runny_nose", "itchy_eyes"], "treatment": "antihistamines", "severity": "mild"},
            "Asthma": {"symptoms": ["shortness_of_breath", "wheezing", "chest_tightness"], 
                       "treatment": "inhalers and medication", "severity": "moderate"},
        }

        self.relationships = {
            "sneezing": ["Common Cold", "Allergy"],
            "runny_nose": ["Common Cold", "Allergy"],
            "itchy_eyes": ["Allergy"],
            "fever": ["Flu", "Pneumonia", "COVID-19"],
            "cough": ["Flu", "Pneumonia", "Tuberculosis", "COVID-19"],
            "body_aches": ["Flu"],
            "shortness_of_breath": ["Pneumonia", "Asthma", "COVID-19"],
            "weight_loss": ["Tuberculosis"],
            "night_sweats": ["Tuberculosis"],
            "wheezing": ["Asthma"],
            "chest_tightness": ["Asthma"],
            "fatigue": ["COVID-19"],
            "loss_of_taste": ["COVID-19"]
        }

        self.patient_history = []  # Stores past diagnoses

    def get_disease(self, symptoms, duration):
        possible = {}
        total_symptoms = len(symptoms)

        for symptom in symptoms:
            if symptom in self.relationships:
                for disease in self.relationships[symptom]:
                    possible[disease] = possible.get(disease, 0) + 1

        # Compute probabilities
        diagnoses = []
        for disease, count in possible.items():
            required_symptoms = self.diseases[disease]["symptoms"]
            match_percentage = (count / len(required_symptoms)) * 100  # Probability
            severity = self.diseases[disease]["severity"]

            # Adjust probability based on symptom duration
            if duration > 10 and disease in ["Tuberculosis", "Pneumonia"]:
                match_percentage += 10  # Increase likelihood for chronic diseases
            elif duration < 3 and disease in ["Common Cold", "Allergy"]:
                match_percentage += 10  # Increase likelihood for short-term diseases

            diagnoses.append((disease, match_percentage, severity))

        # Sort by highest probability
        diagnoses.sort(key=lambda x: x[1], reverse=True)

        return diagnoses

    def update_history(self, diagnosis):
        self.patient_history.append(diagnosis)

    def show_history(self):
        if not self.patient_history:
            print("\nNo past diagnoses found.")
        else:
            print("\n Your Past Diagnoses:")
            for i, record in enumerate(self.patient_history, 1):
                print(f"{i}. {record}")

# Create instance
kb = KnowledgeBase()

print(" Welcome to the Medical Diagnosis System! \n")
print(" Available symptoms:")
print(", ".join(kb.relationships.keys()))
print("\n Example Input: fever, cough, fatigue")
print(" Use underscores (_) instead of spaces (e.g., runny_nose, shortness_of_breath)")

# Get and clean input
user_input = input("\n Enter the symptoms you are experiencing (comma-separated):\n> ").lower().replace(" ", "_").split(',')
symptoms = [s.strip() for s in user_input if s.strip()]

# Ensure symptoms are provided
if not symptoms:
    print(" No symptoms entered. Please restart the program and enter valid symptoms.")
    exit()

# Get symptom duration
while True:
    try:
        duration = int(input("\n How many days have you had these symptoms? (Enter a number):\n> "))
        break
    except ValueError:
        print(" Invalid input. Please enter a number.")

# Debugging: Print symptoms to verify input processing
print("\n Symptoms entered:", symptoms)
print(f" Duration: {duration} days")

# Get possible diagnoses
diagnoses = kb.get_disease(symptoms, duration)

# Output results
if diagnoses:
    print("\n Possible Diagnoses with Probability and Severity:")
    for disease, probability, severity in diagnoses:
        print(f"- {disease}: {probability:.2f}% chance (Severity: {severity})")
        print(f"  Recommended treatment: {kb.diseases[disease]['treatment']}")
        kb.update_history(disease)  # Save diagnosis to history
else:
    print("\n No matching diagnoses found.")

# Ask if user wants to see history
while True:
    history_input = input("\nWould you like to see your past diagnoses? (yes/no):\n> ").strip().lower()
    if history_input in ["yes", "y"]:
        kb.show_history()
        break
    elif history_input in ["no", "n"]:
        print(" Thank you for using the Medical Diagnosis System! Stay healthy! ")
        break
    else:
        print(" Invalid input. Please enter 'yes' or 'no'.")




